In [ ]:
#GPU 할당 확인 코드
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17595594340821530396, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14509932544
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9802372575162041752
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
EPOCHS=50
NUM_CLASSES = 10
    

def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    #normalize 
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train,axis=(0,1,2,3))
    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)

    y_train =  tf.keras.utils.to_categorical(y_train,NUM_CLASSES)
    y_test =  tf.keras.utils.to_categorical(y_test,NUM_CLASSES)

    return x_train, y_train, x_test, y_test

def build_model(): 
    model = models.Sequential()
 
    #1st blocl
    model.add(layers.Conv2D(32, (3,3), padding='same', 
        input_shape=x_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.2))

    #2nd block
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.3))

    #3d block 
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.4))

    #dense  
    model.add(layers.Flatten())
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
    return model

    model.summary()

    
(x_train, y_train, x_test, y_test) = load_data()
model = build_model()
model.compile(loss='categorical_crossentropy', 
            optimizer='RMSprop', 
            metrics=['accuracy'])

170500096/170498071 [==============================] - 3s 0us/step


In [ ]:
#image augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )

datagen.fit(x_train)

In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
import keras
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
#train
batch_size = 64
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), #훈련데이터 뻥튀기
                    epochs=EPOCHS,
                    verbose=1,validation_data=(x_test,y_test))

#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 

Epoch 1/50
  3/782 [..............................] - ETA: 23s - loss: 0.6935 - accuracy: 0.7552

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


782/782 [==============================] - 26s 34ms/step - loss: 0.7560 - accuracy: 0.7396 - val_loss: 0.6016 - val_accuracy: 0.7986
Epoch 2/50
782/782 [==============================] - 26s 34ms/step - loss: 0.7377 - accuracy: 0.7441 - val_loss: 0.6048 - val_accuracy: 0.7993
Epoch 3/50
782/782 [==============================] - 26s 33ms/step - loss: 0.7336 - accuracy: 0.7448 - val_loss: 0.7321 - val_accuracy: 0.7675
Epoch 4/50
782/782 [==============================] - 27s 34ms/step - loss: 0.7136 - accuracy: 0.7540 - val_loss: 0.6332 - val_accuracy: 0.7932
Epoch 5/50
782/782 [==============================] - 27s 34ms/step - loss: 0.6978 - accuracy: 0.7584 - val_loss: 0.6424 - val_accuracy: 0.7909
Epoch 6/50
782/782 [==============================] - 26s 33ms/step - loss: 0.6898 - accuracy: 0.7611 - val_loss: 0.6973 - val_accuracy: 0.7766
Epoch 7/50
782/782 [==============================] - 26s 33ms/step - loss: 0.6849 - accuracy: 0.7651 - val_loss: 0.6045 - val_accuracy: 0.8008
Epo

In [ ]:
#test
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

79/79 [==============================] - 1s 5ms/step - loss: 0.5278 - accuracy: 0.8396

Test result: 83.960 loss: 0.528
